# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 12:00AM,238812,10,8101249,"Snap Medical Industries, LLC",Released
1,Jun 4 2022 1:42PM,238817,21,616802,"NBTY Global, Inc.",Released
2,Jun 4 2022 1:03PM,238816,21,399290288,"NBTY Global, Inc.",Released
3,Jun 4 2022 12:48PM,238815,21,399290086,"NBTY Global, Inc.",Released
4,Jun 4 2022 12:29PM,238814,21,399290400,"NBTY Global, Inc.",Released
5,Jun 3 2022 4:07PM,238810,20,ALUR191472267,Alumier Labs Inc.,Released
6,Jun 3 2022 4:07PM,238810,20,ALUR587852459,Alumier Labs Inc.,Released
7,Jun 3 2022 4:07PM,238810,20,ALU0006981,Alumier Labs Inc.,Released
8,Jun 3 2022 4:07PM,238810,20,ALUR880492178,Alumier Labs Inc.,Released
9,Jun 3 2022 4:07PM,238810,20,ALU0006982,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,238812,Released,1
10,238814,Released,1
11,238815,Released,1
12,238816,Released,1
13,238817,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238812,NaN,NaN,1.0
238814,NaN,NaN,1.0
238815,NaN,NaN,1.0
238816,NaN,NaN,1.0
238817,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238795,0.0,0.0,1.0
238797,0.0,0.0,1.0
238799,1.0,0.0,1.0
238801,0.0,0.0,1.0
238804,0.0,1.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238795,0,0,1
238797,0,0,1
238799,1,0,1
238801,0,0,1
238804,0,1,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238795,0,0,1
1,238797,0,0,1
2,238799,1,0,1
3,238801,0,0,1
4,238804,0,1,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238795,,,1
1,238797,,,1
2,238799,1,,1
3,238801,,,1
4,238804,,1,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC"
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc."
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc."
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc."
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc."
5,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.
12,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc."
13,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation
27,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc."
28,Jun 3 2022 2:56PM,238799,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",,,1
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc.",,,1
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc.",,,1
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc.",,,1
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc.",,,1
5,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,,,7
6,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",,,1
7,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation,,1,13
8,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc.",,,1
9,Jun 3 2022 2:56PM,238799,10,Eywa Pharma Inc.,1,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,,
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc.",1,,
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc.",1,,
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc.",1,,
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc.",1,,
5,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,7,,
6,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",1,,
7,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation,13,1,
8,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc.",1,,
9,Jun 3 2022 2:56PM,238799,10,Eywa Pharma Inc.,1,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,,
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc.",1,,
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc.",1,,
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc.",1,,
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,NaN,NaN
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc.",1,NaN,NaN
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc.",1,NaN,NaN
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc.",1,NaN,NaN
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,0.0,0.0
1,Jun 4 2022 1:42PM,238817,21,"NBTY Global, Inc.",1,0.0,0.0
2,Jun 4 2022 1:03PM,238816,21,"NBTY Global, Inc.",1,0.0,0.0
3,Jun 4 2022 12:48PM,238815,21,"NBTY Global, Inc.",1,0.0,0.0
4,Jun 4 2022 12:29PM,238814,21,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,716415,15,1.0,1.0
16,238797,1,0.0,0.0
18,238795,1,0.0,0.0
20,238810,7,0.0,0.0
21,955262,4,0.0,0.0
22,477607,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,716415,15,1.0,1.0
1,16,238797,1,0.0,0.0
2,18,238795,1,0.0,0.0
3,20,238810,7,0.0,0.0
4,21,955262,4,0.0,0.0
5,22,477607,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,15,1.0,1.0
1,16,1,0.0,0.0
2,18,1,0.0,0.0
3,20,7,0.0,0.0
4,21,4,0.0,0.0
5,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,15.0
1,16,Released,1.0
2,18,Released,1.0
3,20,Released,7.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,20,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,0.0,0.0
Released,15.0,1.0,1.0,7.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,15.0,1.0,1.0,7.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,15.0,1.0,1.0,7.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()